## Decoupling
Given a general multivariable system with transfer function matrix $G_p$, a decoupler attempts to combine with the system to form a diagonal whole.

<img src="../../assets/decoupling.png">

In [1]:
import sympy
sympy.init_printing()

In [2]:
G_p11, G_p12, G_p21, G_p22 = sympy.symbols('G_p11, G_p12, G_p21, G_p22')
G_p = sympy.Matrix([[G_p11, G_p12],[G_p21, G_p22]])
G_p

⎡Gₚ₁₁  Gₚ₁₂⎤
⎢          ⎥
⎣Gₚ₂₁  Gₚ₂₂⎦

### 1. Inverse-based
Wouldn't it be nice if the system didn't have interaction? In other words, we could choose $T$ such that we have this system with the same diagonal elements as the original system but zeros in the off diagonals.

In [3]:
G_s = sympy.Matrix([[G_p11, 0],[0, G_p22]])
G_s

⎡Gₚ₁₁   0  ⎤
⎢          ⎥
⎣ 0    Gₚ₂₂⎦

Recalling that the combination of $T$ and $G_p$ in series is $G_p T$, we can solve for the decoupler directly

$$ G_P T = G_s \therefore T = G_P^{-1} G_s $$

In [5]:
T = G_p.inv()*G_s
T

⎡      Gₚ₁₁⋅Gₚ₂₂             -Gₚ₁₂⋅Gₚ₂₂      ⎤
⎢─────────────────────  ─────────────────────⎥
⎢Gₚ₁₁⋅Gₚ₂₂ - Gₚ₁₂⋅Gₚ₂₁  Gₚ₁₁⋅Gₚ₂₂ - Gₚ₁₂⋅Gₚ₂₁⎥
⎢                                            ⎥
⎢     -Gₚ₁₁⋅Gₚ₂₁              Gₚ₁₁⋅Gₚ₂₂      ⎥
⎢─────────────────────  ─────────────────────⎥
⎣Gₚ₁₁⋅Gₚ₂₂ - Gₚ₁₂⋅Gₚ₂₁  Gₚ₁₁⋅Gₚ₂₂ - Gₚ₁₂⋅Gₚ₂₁⎦

Let's see if that worked:

In [6]:
G_pT = G_p*T
sympy.simplify(G_pT)

⎡Gₚ₁₁   0  ⎤
⎢          ⎥
⎣ 0    Gₚ₂₂⎦

Pros:

* Controller design can be based on open loop model
* Apparent dynamics (what the controller sees) are simple

Cons:

* T is often not physically realisable
* T is complicated

### 2. Zero off-diagonals
A more common strategy is to solve directly for the off-diagonal elements of set equal to zero.

So we just want 

$$G_P T = \begin{bmatrix}d_1&0\\0&d_2\end{bmatrix}$$

Note the difference between the first method and this one - here we are not specifying the diagonal at all, we just want the off-diagonals to be zero.

In [7]:
T21, T12 = sympy.symbols('T21, T12')
T = sympy.Matrix([[1, T12], 
                  [T21, 1]])

wantdiagonal = G_p*T

sol = sympy.solve([wantdiagonal[0,1], wantdiagonal[1, 0]], [T21, T12])

In [8]:
T.subs(sol)

⎡        -Gₚ₁₂ ⎤
⎢  1     ──────⎥
⎢         Gₚ₁₁ ⎥
⎢              ⎥
⎢-Gₚ₂₁         ⎥
⎢──────    1   ⎥
⎣ Gₚ₂₂         ⎦

So this is the classic/traditional decoupler shown in the diagram (with unit passthrough on the diagonals). This changes the transfer function the controller "sees" to

In [9]:
G_p*T.subs(sol)

⎡       Gₚ₁₂⋅Gₚ₂₁                  ⎤
⎢Gₚ₁₁ - ─────────         0        ⎥
⎢          Gₚ₂₂                    ⎥
⎢                                  ⎥
⎢                         Gₚ₁₂⋅Gₚ₂₁⎥
⎢       0          Gₚ₂₂ - ─────────⎥
⎣                            Gₚ₁₁  ⎦

Pros:

* Relatively simple design process
* Less complicated decoupler than the inverse-based method

Cons:

* Apparent plant may be higher order than the actual plant
* Still requires an inverse, may not be physically realisable (but more likely than method 1)

### 3. Adjugate method
The adjugate (previously calld the adjoint) of a matrix will also diagonalise a system

In [10]:
T = G_p.adjugate()
T

⎡Gₚ₂₂   -Gₚ₁₂⎤
⎢            ⎥
⎣-Gₚ₂₁  Gₚ₁₁ ⎦

In [11]:
G_p*T

⎡Gₚ₁₁⋅Gₚ₂₂ - Gₚ₁₂⋅Gₚ₂₁            0          ⎤
⎢                                            ⎥
⎣          0            Gₚ₁₁⋅Gₚ₂₂ - Gₚ₁₂⋅Gₚ₂₁⎦

Pros:
    
* Decoupler guaranteed to be physically realisable because it only requires "forward" models of the system.

Cons:

* Apparent plant now much higher order (look at the products in the $G_pT$ expression)